In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)   # para visualizar todas las columnas del df
#pd.set_option('display.max_rows', None)   # para visualizar todas las filas del df

import mysql.connector as conn

from selenium import webdriver
import time

In [2]:
def crear_db(nombre, user='root', password="Bestjob1_", check=True):
    
    crea_db=conn.connect(host='localhost', user=user, passwd=password)

    cursor=crea_db.cursor()

    cursor.execute(f'drop database if exists {nombre}')
    cursor.execute(f'create database {nombre}')
    
    print('Hecho, base de datos creada. ;)')
    
    if check:
        cursor.execute('show databases')
        for x in cursor:
            print(x)
#MySQL Bestjob1_

In [3]:
def crear_tabla_desde_df(df, nombre_tabla, nombre_db, user='root', password='Bestjob1_', check=True, drop=False):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    if drop:
        cursor.execute(f'drop table if exists {nombre_tabla}')
    
    tabla=f"create table {nombre_tabla}(id int, {' varchar(200),'.join(df.columns)} varchar(200));"
    print(tabla)
    
    cursor.execute(tabla)
    
    print(f'Hecho, tabla {nombre_tabla} creada. ;)')
    
    if check:
        cursor.execute('show tables')
        for x in cursor:
            print(x)

In [4]:
def insertar_datos(df, nombre_tabla, nombre_db, user='root', password='Bestjob1_', check=True):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    for i in range(len(df)):
    
        insert_query='insert into {} (id, {}) values {};'\
                     .format(nombre_tabla, ','.join(df.columns), tuple(np.insert(df.iloc[i].values, 0, i)))

        cursor.execute(insert_query)

    db.commit()
    
    print(f'Hecho, datos dentro de tabla {nombre_tabla}. ;)')
    print()
    print(insert_query)

In [5]:
def leer_sql(query, nombre_tabla, nombre_db, user='root', password='Bestjob1_'):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    
    data=pd.read_sql(query, db)
    
    return data

In [6]:
##Primera BD consumo de alimentos por paises

In [7]:
df=pd.read_csv("FAO.csv")
df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [8]:
df.shape

(21477, 63)

In [9]:
df.columns

Index(['Area Abbreviation', 'Area Code', 'Area', 'Item Code', 'Item',
       'Element Code', 'Element', 'Unit', 'latitude', 'longitude', 'Y1961',
       'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969',
       'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977',
       'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985',
       'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993',
       'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013'],
      dtype='object')

In [10]:
df=df.drop(columns=["Y1961",'Y1962','Y1963','Y1964','Y1965', 'Y1966','Y1967','Y1968','Y1969','Y1971','Y1972','Y1973','Y1974','Y1975','Y1976',"Y1977","Y1978",'Y1979','Y1981','Y1982','Y1983','Y1984','Y1985','Y1986',"Y1987","Y1988",'Y1989','Y1991','Y1992','Y1993','Y1994','Y1995','Y1996','Y1997','Y1998','Y1999'])

In [11]:
df.shape

(21477, 27)

In [12]:
df.columns

Index(['Area Abbreviation', 'Area Code', 'Area', 'Item Code', 'Item',
       'Element Code', 'Element', 'Unit', 'latitude', 'longitude', 'Y1970',
       'Y1980', 'Y1990', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005',
       'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013'],
      dtype='object')

In [13]:
df=df.drop(columns=["Area Abbreviation", "Area Code","Item Code","Element Code","Element","latitude","longitude"])

In [14]:
df.columns

Index(['Area', 'Item', 'Unit', 'Y1970', 'Y1980', 'Y1990', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013'],
      dtype='object')

In [15]:
df.head()

,Area,Item,Unit,Y1970,Y1980,Y1990,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,Afghanistan,Wheat and products,1000 tonnes,1819.0,2129.0,1640.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,Afghanistan,Rice (Milled Equivalent),1000 tonnes,213.0,259.0,199.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,Afghanistan,Barley and products,1000 tonnes,74.0,64.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,Afghanistan,Barley and products,1000 tonnes,234.0,202.0,135.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,Afghanistan,Maize and products,1000 tonnes,200.0,226.0,144.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [16]:
#Solamente quiero los datos de consumo de carne
df.Item=df.Item.str.lower().str.strip()
df.Item=df.Item.str.split("(").str[0]
df.Item=df.Item.str.split("-").str[0]
df.loc[df["Item"].str.contains("meat", case=False), "Item"] = "meat"
df.loc[df["Item"].str.contains("fat", case=False), "Item"] = "meat"
df.loc[df["Item"].str.contains("animal", case=False), "Item"] = "meat"

In [17]:
df.Item.value_counts()

meat             1670
milk              558
eggs              360
cereals           347
fish, seafood     337
                 ... 
sugar beet         66
sugar non          32
palm kernels       24
cottonseed         21
ricebran oil       18
Name: Item, Length: 106, dtype: int64

In [18]:
df.drop(df[df['Item'] !="meat"].index, inplace = True)

In [19]:
df.Item.value_counts()

meat    1670
Name: Item, dtype: int64

In [20]:
df=df.groupby(["Area","Item"]).sum()

In [21]:
df=df.reset_index()

In [22]:
df.Area.value_counts()

Liberia              1
Uganda               1
Malawi               1
Yemen                1
Slovenia             1
                    ..
Cabo Verde           1
Bermuda              1
Grenada              1
Lebanon              1
Republic of Korea    1
Name: Area, Length: 174, dtype: int64

In [23]:
df.columns=df.columns.str.lower()
df=df.rename(columns={"area":"country","item":"food"})

In [24]:
#df final total consumo de carne por pais

In [25]:
df.shape

(174, 19)

In [26]:
df.head()

,country,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013
0,Afghanistan,meat,418.0,524.0,532.0,703.0,605.0,667.0,652.0,718.0,756.0,628.0,620.0,715.0,719.0,805.0,848.0,804,838
1,Albania,meat,84.0,96.0,136.0,190.0,221.0,260.0,260.0,271.0,276.0,299.0,328.0,356.0,374.0,390.0,386.0,404,406
2,Algeria,meat,251.0,479.0,978.0,1193.0,1106.0,1172.0,1269.0,1417.0,1451.0,1389.0,1399.0,1384.0,1438.0,1400.0,1547.0,1655,1691
3,Angola,meat,130.0,230.0,310.0,481.0,480.0,606.0,681.0,624.0,740.0,819.0,896.0,1005.0,965.0,1123.0,1323.0,1482,1579
4,Antigua and Barbuda,meat,5.0,6.0,10.0,11.0,11.0,11.0,11.0,12.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0,15,15


In [27]:
!pip install mysql-connector-python

You should consider upgrading via the 'c:\users\atefe\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [28]:
pip install mysqlclient

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\atefe\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [29]:
crear_db("proyectoetl", user='root', password="Bestjob1_", check=True)

Hecho, base de datos creada. ;)
('information_schema',)
('lab_mysql',)
('mysql',)
('newschema',)
('performance_schema',)
('proyectoetl',)
('publications',)
('shark_cleaned',)
('sys',)


In [30]:
crear_tabla_desde_df(df, 'food', 'proyectoetl')

create table food(id int, country varchar(200),food varchar(200),y1970 varchar(200),y1980 varchar(200),y1990 varchar(200),y2000 varchar(200),y2001 varchar(200),y2002 varchar(200),y2003 varchar(200),y2004 varchar(200),y2005 varchar(200),y2006 varchar(200),y2007 varchar(200),y2008 varchar(200),y2009 varchar(200),y2010 varchar(200),y2011 varchar(200),y2012 varchar(200),y2013 varchar(200));
Hecho, tabla food creada. ;)
('food',)


In [31]:
insertar_datos(df, 'food', 'proyectoetl')

Hecho, datos dentro de tabla food. ;)

insert into food (id, country,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013) values (173, 'Zimbabwe', 'meat', 151.0, 217.0, 315.0, 388.0, 484.0, 437.0, 465.0, 493.0, 505.0, 521.0, 530.0, 535.0, 588.0, 580.0, 599.0, 604, 552);


In [32]:
leer_sql('select * from food limit 3', 'food', 'proyectoetl')

,id,country,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013
0,0,Afghanistan,meat,418.0,524.0,532.0,703.0,605.0,667.0,652.0,718.0,756.0,628.0,620.0,715.0,719.0,805.0,848.0,804,838
1,1,Albania,meat,84.0,96.0,136.0,190.0,221.0,260.0,260.0,271.0,276.0,299.0,328.0,356.0,374.0,390.0,386.0,404,406
2,2,Algeria,meat,251.0,479.0,978.0,1193.0,1106.0,1172.0,1269.0,1417.0,1451.0,1389.0,1399.0,1384.0,1438.0,1400.0,1547.0,1655,1691


In [33]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [34]:
webdriver.Chrome(ChromeDriverManager().install()).get('https://www.google.es')



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\atefe\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [35]:
PATH=ChromeDriverManager().install()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\atefe\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [36]:
#Segunda dt emision de CO2

In [37]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_carbon_dioxide_emissions_per_capita"

driver=webdriver.Chrome(PATH)
driver.get(url)
time.sleep(2) 

cabeceras=driver.find_element_by_tag_name('thead')
cabeceras=[c.text for c in cabeceras.find_elements_by_tag_name('th')]

tabla=driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/table[2]/tbody')

filas=tabla.find_elements_by_tag_name('tr')

datos=[[e.text.strip() for e in f.find_elements_by_tag_name('td')] for f in filas]


In [38]:
o2emission=pd.DataFrame(datos, columns=cabeceras)

In [39]:
o2emission.head()

,,1980,1985,1990,1995,2000,2008,2009,2010,2011,2012,2013,2014,2015,2018
0,Afghanistan,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3
1,Albania,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6
2,Algeria,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9
3,American Samoa,..,..,..,..,..,..,..,0.6,0.7,0.7,0.7,0.7,0.8,..
4,Andorra,..,..,7.5,6.7,8.0,6.4,6.1,6.1,5.9,5.9,5.9,5.8,5.9,6.0


In [40]:
o2emission.columns=o2emission.columns.str.lower()

In [41]:
o2emission["o2emission"]="o2emission"

In [42]:
o2emission.columns

Index(['', '1980', '1985', '1990', '1995', '2000', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2018', 'o2emission'],
      dtype='object')

In [43]:
o2emission=o2emission.rename(columns={"":"country"})

In [44]:
o2emission= o2emission[['country', 'o2emission', '1980','1985','1990','1995','2000','2008','2009','2010','2011','2012','2013','2014','2015','2018']]

In [45]:
o2emission= o2emission.rename(columns={"1980":'yr1980','1985':'yr1985','1990':'yr1990','1995':'yr1995','2000':'yr2000','2008':'yr2008','2009':'yr2009','2010':'yr2010',"2011":'yr2011','2012':'yr2012','2013':'yr2013','2014':'yr2014','2015':'yr2015','2018':'yr2018'})

In [46]:
#DF final con la emision por pais

In [47]:
o2emission.head()

,country,o2emission,yr1980,yr1985,yr1990,yr1995,yr2000,yr2008,yr2009,yr2010,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018
0,Afghanistan,o2emission,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3
1,Albania,o2emission,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6
2,Algeria,o2emission,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9
3,American Samoa,o2emission,..,..,..,..,..,..,..,0.6,0.7,0.7,0.7,0.7,0.8,..
4,Andorra,o2emission,..,..,7.5,6.7,8.0,6.4,6.1,6.1,5.9,5.9,5.9,5.8,5.9,6.0


In [48]:
o2emission.shape

(223, 16)

In [49]:
crear_tabla_desde_df(o2emission , "o2emissionyear", "proyectoetl")

create table o2emissionyear(id int, country varchar(200),o2emission varchar(200),yr1980 varchar(200),yr1985 varchar(200),yr1990 varchar(200),yr1995 varchar(200),yr2000 varchar(200),yr2008 varchar(200),yr2009 varchar(200),yr2010 varchar(200),yr2011 varchar(200),yr2012 varchar(200),yr2013 varchar(200),yr2014 varchar(200),yr2015 varchar(200),yr2018 varchar(200));
Hecho, tabla o2emissionyear creada. ;)
('food',)
('o2emissionyear',)


In [50]:
insertar_datos(o2emission , "o2emissionyear", "proyectoetl")

Hecho, datos dentro de tabla o2emissionyear. ;)

insert into o2emissionyear (id, country,o2emission,yr1980,yr1985,yr1990,yr1995,yr2000,yr2008,yr2009,yr2010,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018) values (222, 'Zimbabwe', 'o2emission', '1.3', '1.2', '1.5', '1.3', '1.1', '0.6', '0.4', '0.6', '0.7', '0.5', '0.8', '0.8', '0.8', '0.8');


In [51]:
leer_sql('select * from o2emissionyear limit 3', 'o2emissionyear', 'proyectoetl')

,id,country,o2emission,yr1980,yr1985,yr1990,yr1995,yr2000,yr2008,yr2009,yr2010,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018
0,0,Afghanistan,o2emission,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3
1,1,Albania,o2emission,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6
2,2,Algeria,o2emission,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9


In [52]:
!pip install fuzzywuzzy

You should consider upgrading via the 'c:\users\atefe\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [53]:
from fuzzywuzzy import process, fuzz

c:\users\atefe\appdata\local\programs\python\python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [54]:
df_food=leer_sql('select * from food', 'food', 'proyectoetl')
df_o2emission=leer_sql('select * from o2emissionyear', 'o2emissionyear', 'proyectoetl')

In [56]:
fusionado=pd.merge(df_o2emission, df_food, left_on='country', right_on='country')

In [57]:
#fusionado consumo de carne y emision de CO2 por paises

In [58]:
fusionado.head()

,id_x,country,o2emission,yr1980,yr1985,yr1990,yr1995,yr2000,yr2008,yr2009,yr2010,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018,id_y,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013
0,0,Afghanistan,o2emission,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0,meat,418.0,524.0,532.0,703.0,605.0,667.0,652.0,718.0,756.0,628.0,620.0,715.0,719.0,805.0,848.0,804,838
1,1,Albania,o2emission,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6,1,meat,84.0,96.0,136.0,190.0,221.0,260.0,260.0,271.0,276.0,299.0,328.0,356.0,374.0,390.0,386.0,404,406
2,2,Algeria,o2emission,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9,2,meat,251.0,479.0,978.0,1193.0,1106.0,1172.0,1269.0,1417.0,1451.0,1389.0,1399.0,1384.0,1438.0,1400.0,1547.0,1655,1691
3,5,Angola,o2emission,0.6,0.4,0.4,0.8,0.6,1.2,1.2,1.2,1.3,1.3,1.3,1.3,1.2,1.0,3,meat,130.0,230.0,310.0,481.0,480.0,606.0,681.0,624.0,740.0,819.0,896.0,1005.0,965.0,1123.0,1323.0,1482,1579
4,6,Antigua and Barbuda,o2emission,1.9,3.6,4.2,3.7,4.1,5.2,5.4,5.5,5.4,5.4,5.4,5.4,6.2,6.2,4,meat,5.0,6.0,10.0,11.0,11.0,11.0,11.0,12.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0,15,15


In [59]:
fusionado=fusionado.drop(["id_x"], axis=1)
fusionado=fusionado.drop(["id_y"], axis=1)

In [60]:
fusionado.head()

,country,o2emission,yr1980,yr1985,yr1990,yr1995,yr2000,yr2008,yr2009,yr2010,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013
0,Afghanistan,o2emission,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3,meat,418.0,524.0,532.0,703.0,605.0,667.0,652.0,718.0,756.0,628.0,620.0,715.0,719.0,805.0,848.0,804,838
1,Albania,o2emission,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6,meat,84.0,96.0,136.0,190.0,221.0,260.0,260.0,271.0,276.0,299.0,328.0,356.0,374.0,390.0,386.0,404,406
2,Algeria,o2emission,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9,meat,251.0,479.0,978.0,1193.0,1106.0,1172.0,1269.0,1417.0,1451.0,1389.0,1399.0,1384.0,1438.0,1400.0,1547.0,1655,1691
3,Angola,o2emission,0.6,0.4,0.4,0.8,0.6,1.2,1.2,1.2,1.3,1.3,1.3,1.3,1.2,1.0,meat,130.0,230.0,310.0,481.0,480.0,606.0,681.0,624.0,740.0,819.0,896.0,1005.0,965.0,1123.0,1323.0,1482,1579
4,Antigua and Barbuda,o2emission,1.9,3.6,4.2,3.7,4.1,5.2,5.4,5.5,5.4,5.4,5.4,5.4,6.2,6.2,meat,5.0,6.0,10.0,11.0,11.0,11.0,11.0,12.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0,15,15


In [61]:
crear_tabla_desde_df(fusionado, 'fusion', 'proyectoetl', drop=True)
insertar_datos(fusionado, 'fusion', 'proyectoetl')

leer_sql('select * from fusion limit 3', 'fusion', 'proyectoetl')

create table fusion(id int, country varchar(200),o2emission varchar(200),yr1980 varchar(200),yr1985 varchar(200),yr1990 varchar(200),yr1995 varchar(200),yr2000 varchar(200),yr2008 varchar(200),yr2009 varchar(200),yr2010 varchar(200),yr2011 varchar(200),yr2012 varchar(200),yr2013 varchar(200),yr2014 varchar(200),yr2015 varchar(200),yr2018 varchar(200),food varchar(200),y1970 varchar(200),y1980 varchar(200),y1990 varchar(200),y2000 varchar(200),y2001 varchar(200),y2002 varchar(200),y2003 varchar(200),y2004 varchar(200),y2005 varchar(200),y2006 varchar(200),y2007 varchar(200),y2008 varchar(200),y2009 varchar(200),y2010 varchar(200),y2011 varchar(200),y2012 varchar(200),y2013 varchar(200));
Hecho, tabla fusion creada. ;)
('food',)
('fusion',)
('o2emissionyear',)
Hecho, datos dentro de tabla fusion. ;)

insert into fusion (id, country,o2emission,yr1980,yr1985,yr1990,yr1995,yr2000,yr2008,yr2009,yr2010,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2

,id,country,o2emission,yr1980,yr1985,yr1990,yr1995,yr2000,yr2008,yr2009,yr2010,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013
0,0,Afghanistan,o2emission,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3,meat,418.0,524.0,532.0,703.0,605.0,667.0,652.0,718.0,756.0,628.0,620.0,715.0,719.0,805.0,848.0,804,838
1,1,Albania,o2emission,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6,meat,84.0,96.0,136.0,190.0,221.0,260.0,260.0,271.0,276.0,299.0,328.0,356.0,374.0,390.0,386.0,404,406
2,2,Algeria,o2emission,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9,meat,251.0,479.0,978.0,1193.0,1106.0,1172.0,1269.0,1417.0,1451.0,1389.0,1399.0,1384.0,1438.0,1400.0,1547.0,1655,1691


In [62]:
#forest area reducction per country

In [63]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_forest_area"

driver=webdriver.Chrome(PATH)
driver.get(url)
time.sleep(2) 

cabeceras=driver.find_element_by_tag_name('thead')
cabeceras=[c.text for c in cabeceras.find_elements_by_tag_name('th')]

tabla=driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/table[2]')

filas=tabla.find_elements_by_tag_name('tr')

datos=[[e.text.strip() for e in f.find_elements_by_tag_name('td')] for f in filas]

In [64]:
cabeceras

['Region', '1990', '2000', '2010', '2020']

In [65]:
forest=pd.DataFrame(datos, columns=cabeceras)

In [66]:
forest=forest.iloc[1: , :]

In [77]:
forest

,country,yr1990,yr2000,yr2010,yr2020,forest
1,Afghanistan,"1,208","1,208","1,208","1,208",forest
2,Albania,789,769,782,789,forest
3,Algeria,"1,667","1,579","1,918","1,949",forest
4,American Samoa,18,18,17,17,forest
5,Andorra,16,16,16,16,forest
...,...,...,...,...,...,...
190,Venezuela,"52,026","49,151","47,505","46,231",forest
191,Vietnam,"9,376","11,784","13,388","14,643",forest
192,Yemen,549,549,549,549,forest
193,Zambia,"47,412","47,054","46,696","44,814",forest


In [78]:
forest=forest.rename(columns={"Region":"country","1990":"yr1990","2000":"yr2000","2010":"yr2010","2020":"yr2020"},)

In [79]:
forest["forest"]="forest"

In [80]:
forest.head()

,country,yr1990,yr2000,yr2010,yr2020,forest
1,Afghanistan,"1,208","1,208","1,208","1,208",forest
2,Albania,789,769,782,789,forest
3,Algeria,"1,667","1,579","1,918","1,949",forest
4,American Samoa,18,18,17,17,forest
5,Andorra,16,16,16,16,forest


In [81]:
forest= forest[["country","forest","yr1990","yr2000","yr2010","yr2020"]]

In [82]:
forest.head()

,country,forest,yr1990,yr2000,yr2010,yr2020
1,Afghanistan,forest,"1,208","1,208","1,208","1,208"
2,Albania,forest,789,769,782,789
3,Algeria,forest,"1,667","1,579","1,918","1,949"
4,American Samoa,forest,18,18,17,17
5,Andorra,forest,16,16,16,16


In [83]:
crear_tabla_desde_df(forest , "forest", "proyectoetl")

create table forest(id int, country varchar(200),forest varchar(200),yr1990 varchar(200),yr2000 varchar(200),yr2010 varchar(200),yr2020 varchar(200));
Hecho, tabla forest creada. ;)
('food',)
('forest',)
('fusion',)
('o2emissionyear',)


In [84]:
insertar_datos(forest , "forest", "proyectoetl")

Hecho, datos dentro de tabla forest. ;)

insert into forest (id, country,forest,yr1990,yr2000,yr2010,yr2020) values (193, 'Zimbabwe', 'forest', '18,827', '18,366', '17,905', '17,445');


In [85]:
leer_sql('select * from forest limit 3', 'forest', 'proyectoetl')

,id,country,forest,yr1990,yr2000,yr2010,yr2020
0,0,Afghanistan,forest,"1,208","1,208","1,208","1,208"
1,1,Albania,forest,789,769,782,789
2,2,Algeria,forest,"1,667","1,579","1,918","1,949"


In [86]:
#fusion del ultimo Df con otras dos

In [87]:
df_fusion=leer_sql('select * from fusion', 'fusion', 'proyectoetl')
df_forest=leer_sql('select * from forest', 'forest', 'proyectoetl')

In [88]:
total=pd.merge(df_fusion, df_forest, left_on='country', right_on='country')

In [89]:
total.head()

,id_x,country,o2emission,yr1980,yr1985,yr1990_x,yr1995,yr2000_x,yr2008,yr2009,yr2010_x,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013,id_y,forest,yr1990_y,yr2000_y,yr2010_y,yr2020
0,0,Afghanistan,o2emission,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3,meat,418.0,524.0,532.0,703.0,605.0,667.0,652.0,718.0,756.0,628.0,620.0,715.0,719.0,805.0,848.0,804,838,0,forest,"1,208","1,208","1,208","1,208"
1,1,Albania,o2emission,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6,meat,84.0,96.0,136.0,190.0,221.0,260.0,260.0,271.0,276.0,299.0,328.0,356.0,374.0,390.0,386.0,404,406,1,forest,789,769,782,789
2,2,Algeria,o2emission,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9,meat,251.0,479.0,978.0,1193.0,1106.0,1172.0,1269.0,1417.0,1451.0,1389.0,1399.0,1384.0,1438.0,1400.0,1547.0,1655,1691,2,forest,"1,667","1,579","1,918","1,949"
3,3,Angola,o2emission,0.6,0.4,0.4,0.8,0.6,1.2,1.2,1.2,1.3,1.3,1.3,1.3,1.2,1.0,meat,130.0,230.0,310.0,481.0,480.0,606.0,681.0,624.0,740.0,819.0,896.0,1005.0,965.0,1123.0,1323.0,1482,1579,5,forest,"79,263","77,709","72,158","66,607"
4,4,Antigua and Barbuda,o2emission,1.9,3.6,4.2,3.7,4.1,5.2,5.4,5.5,5.4,5.4,5.4,5.4,6.2,6.2,meat,5.0,6.0,10.0,11.0,11.0,11.0,11.0,12.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0,15,15,6,forest,10,9,9,8


In [90]:
total=total.drop(["id_x"], axis=1)
total=total.drop(["id_y"], axis=1)

In [91]:
total.head()

,country,o2emission,yr1980,yr1985,yr1990_x,yr1995,yr2000_x,yr2008,yr2009,yr2010_x,yr2011,yr2012,yr2013,yr2014,yr2015,yr2018,food,y1970,y1980,y1990,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013,forest,yr1990_y,yr2000_y,yr2010_y,yr2020
0,Afghanistan,o2emission,0.1,0.3,0.2,0.1,0.0,0.2,0.2,0.3,0.4,0.4,0.3,0.3,0.3,0.3,meat,418.0,524.0,532.0,703.0,605.0,667.0,652.0,718.0,756.0,628.0,620.0,715.0,719.0,805.0,848.0,804,838,forest,"1,208","1,208","1,208","1,208"
1,Albania,o2emission,1.9,2.7,1.7,0.7,1.0,1.5,1.5,1.6,1.8,1.7,1.7,2.0,1.6,1.6,meat,84.0,96.0,136.0,190.0,221.0,260.0,260.0,271.0,276.0,299.0,328.0,356.0,374.0,390.0,386.0,404,406,forest,789,769,782,789
2,Algeria,o2emission,3.4,3.2,3.0,3.3,2.8,3.2,3.4,3.3,3.3,3.5,3.5,3.7,3.9,3.9,meat,251.0,479.0,978.0,1193.0,1106.0,1172.0,1269.0,1417.0,1451.0,1389.0,1399.0,1384.0,1438.0,1400.0,1547.0,1655,1691,forest,"1,667","1,579","1,918","1,949"
3,Angola,o2emission,0.6,0.4,0.4,0.8,0.6,1.2,1.2,1.2,1.3,1.3,1.3,1.3,1.2,1.0,meat,130.0,230.0,310.0,481.0,480.0,606.0,681.0,624.0,740.0,819.0,896.0,1005.0,965.0,1123.0,1323.0,1482,1579,forest,"79,263","77,709","72,158","66,607"
4,Antigua and Barbuda,o2emission,1.9,3.6,4.2,3.7,4.1,5.2,5.4,5.5,5.4,5.4,5.4,5.4,6.2,6.2,meat,5.0,6.0,10.0,11.0,11.0,11.0,11.0,12.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0,15,15,forest,10,9,9,8


In [93]:
total.shape

(146, 39)

In [ ]:
#Recycling rates per country

In [ ]:
#url = "https://en.wikipedia.org/wiki/List_of_countries_by_forest_area"